<a href="https://colab.research.google.com/github/lydiasu1/pythonanalysis/blob/master/Pythonanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lien de la dataset : https://archive.ics.uci.edu/ml/datasets/PPG-DaLiA

PPG-DaLiA contains data from 15 subjects wearing physiological and motion sensors, providing a PPG dataset for motion compensation and heart rate estimation in Daily Life Activities.

Objectif : Classification -> Prédire l'activité de la personne

Enregistrement des données sur 15 personnes
Informations de ces 15 personnes dans SX_quest.csv
Informations sur les activités et leur durées : Table 2


A faire : 
- Comprendre comment utiliser rpeaks

Faites : 


In [3]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/00495/data.zip --output data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2731M  100 2731M    0     0  72.3M      0  0:00:37  0:00:37 --:--:-- 73.5M


In [4]:
!unzip data

Archive:  data
   creating: PPG_FieldStudy/
  inflating: PPG_FieldStudy/PPG_FieldStudy_readme.pdf  
   creating: PPG_FieldStudy/S1/
   creating: PPG_FieldStudy/S10/
  inflating: PPG_FieldStudy/S10/S10.pkl  
  inflating: PPG_FieldStudy/S10/S10_activity.csv  
  inflating: PPG_FieldStudy/S10/S10_E4.zip  
  inflating: PPG_FieldStudy/S10/S10_quest.csv  
  inflating: PPG_FieldStudy/S10/S10_RespiBAN.h5  
   creating: PPG_FieldStudy/S11/
  inflating: PPG_FieldStudy/S11/S11.pkl  
  inflating: PPG_FieldStudy/S11/S11_activity.csv  
  inflating: PPG_FieldStudy/S11/S11_E4.zip  
  inflating: PPG_FieldStudy/S11/S11_quest.csv  
  inflating: PPG_FieldStudy/S11/S11_RespiBAN.h5  
   creating: PPG_FieldStudy/S12/
  inflating: PPG_FieldStudy/S12/S12.pkl  
  inflating: PPG_FieldStudy/S12/S12_activity.csv  
  inflating: PPG_FieldStudy/S12/S12_E4.zip  
  inflating: PPG_FieldStudy/S12/S12_quest.csv  
  inflating: PPG_FieldStudy/S12/S12_RespiBAN.h5  
   creating: PPG_FieldStudy/S13/
  inflating: PPG_FieldStudy/

# **Compréhension de la data**
Je réalise des analyses que sur le sujet 1 pour comprendre le sens de la donnée

In [0]:
import pandas as pd

df= pd.read_pickle("./PPG_FieldStudy/S1/S1.pkl")

In [0]:
#Organisation de la data
print(f"La data est organisée en différentes catégories : {list(df.keys())}")

#ID du sujet
print(f"Il s'agit du sujet : {df['subject']}")

#Caractéristiques du sujet (Poids, genre, age, taille, couleur de peau, niveau de sportivité)

print(f"Les caractéristiques du sujet sont les suivantes : {df['questionnaire']}")

#Les activités sont sous forme d'array
print(f'Activités : {df["activity"]} \n Taille de l\'array : {len(df["activity"])}')

## Activity :

In [0]:
def plot_activity(data,name_column):
  import matplotlib.pyplot as plt
  import seaborn as sns
  import numpy as np
  sns.set_style('whitegrid')
  unique, counts = np.unique(data[name_column], return_counts=True)
  dict(zip(unique, counts))
  plt.figure(figsize=(10,5))
  sns.barplot(unique, counts, alpha=0.8)
  for i in range(len(unique)):
    plt.text(x = unique[i]-0.2 , y = counts[i]+10, s = counts[i], size = 10)
  plt.title('Visualisation de la répartition des activités')
  plt.ylabel("Nombres d'occurences", fontsize=12)
  plt.xlabel('Numéro des activités', fontsize=12)
  plt.show()

In [0]:
plot_activity(df,'activity')
#A voir après si on conserve les numéros ou on les remplace par leur noms

In [0]:
activity = pd.DataFrame(df['activity'],columns=['activity'])
activity.plot(y='activity', use_index=True)

D'après l'histogramme, les activités les plus réalisées par notre sujet 1 est l'acitvité 0 c'est à dire la période de transition entre les activités et l'activité 6 qui est la pause déjeuner

On voit bien dans le deuxième graphique les périodes de transition (Quand activité = 0) et on voit bien que l'activité 6 a pris un temps plus important que les autres activités

##Rpeaks : 

In [0]:
import numpy as np
unique, counts = np.unique(df['rpeaks'], return_counts=True)
dico_rpeaks = sorted(dict(zip(unique, counts)))
#Ce sont des index donc normalement valeurs uniques
print('En enlevant les doublons : ',len(set(df['rpeaks'])),'\nSans doublons :  ',len(df['rpeaks']))
print(f'Les valeurs de rpeaks pour le sujet 1 sont comprises entre {min(dico_rpeaks)} et {max(dico_rpeaks)}.La taille de l array est de {len(df["rpeaks"])}')

#D'après le readme, rpeaks correspond aux index qui référent aux EGC signal inclus dans label

## Label : 

In [0]:
print(f"""Label est de type {type(df['label'])}.
Les valeurs sont de type {type(df['label'][0])}
Son minimum est {min(df['label'])}
Son maximum est {max(df['label'])}
La taille de l'array est de {len(df['label'])}
""")

In [0]:
label = pd.DataFrame(df['label'],columns=['labels'])
label
label.plot(y='labels', use_index=True)


Nous avons fait un plot représentant les données dans de la colonne label. Il s'agit de la représentation dans le temps du ground truth heart rate information.

## Lien etre label et activity :

In [0]:
taille_activite = len(df['activity'])
taille_label = len(df['label'])
print(f"""

Taille de la data activité : {taille_activite}
Taille de la data label    : {taille_label}

Dans les données, ils nous parlent de windows de 8s. Ils ont fait une moyenne des battements de coeurs tous les 8 secondes.

Multiplication des deux variables : {taille_label*8}

Différence est de : {taille_activite - taille_label*8}

""")

In [0]:
import matplotlib.pyplot as plt

plt.plot(label.index*8, label['labels'])

#Multiplier par 20 pour l'échelle
plt.plot(activity.index,activity['activity']*20)
plt.show()

On peut remarquer des segmentations similaires c'est à dire que les battements de coeur ont l'air de changer selon l'activité

##Signal :

In [0]:
print(f"Dans signal, nous avons un dataset en {len(df['signal'].keys())} parties : {list(df['signal'].keys())})")
print(f"""

Dans chest, nous avons la structure suivante : {list(df['signal'].get('chest').keys())}
Dans wrist, nous avons la structure suivante : {list(df['signal'].get('wrist').keys())}

""")

In [0]:
#Divisé en deux : Chest et wrist
df_chest = pd.DataFrame()
for i in df.get('signal').get('chest').keys():
  column = df.get('signal').get('chest').get(i)
  df_chest[i] = column.tolist()

In [0]:
for i in df_chest.keys():
  if (i == 'ACC'):
    acc1 = df_chest[i].apply(lambda x: x[0])
    acc2 = df_chest[i].apply(lambda x: x[1])
    acc3 = df_chest[i].apply(lambda x: x[2])
    df_chest['ACC1'] = acc1
    df_chest['ACC2'] = acc2
    df_chest['ACC3'] = acc3
  else:
    new_data = df_chest[i].apply(lambda x: x[0])
    df_chest[i]=new_data

ACC : 3 dimensions
ECG :
EMG : 
DA :
Temp : 
Resp :

In [25]:
df_chest.head()

,ACC,ECG,EMG,EDA,Temp,Resp,ACC1,ACC2,ACC3
0,"[0.8564000129699707, -0.06779998540878296, -0....",0.015610,-1.5,0.0,-273.149994,4.441833,0.8564,-0.0678,-0.3656
1,"[0.8555999994277954, -0.0690000057220459, -0.3...",0.018127,-1.5,0.0,-273.149994,4.450989,0.8556,-0.0690,-0.3646
2,"[0.8519999980926514, -0.07020002603530884, -0....",0.017532,-1.5,0.0,-273.149994,4.473877,0.8520,-0.0702,-0.3642
3,"[0.8525999784469604, -0.0690000057220459, -0.3...",0.013412,-1.5,0.0,-273.149994,4.478455,0.8526,-0.0690,-0.3640
4,"[0.8525999784469604, -0.06980001926422119, -0....",0.011948,-1.5,0.0,-273.149994,4.510498,0.8526,-0.0698,-0.3654


In [0]:
df_chest.drop('ACC',axis=1)

In [27]:
df_chest.describe()

,ECG,EMG,EDA,Temp,Resp,ACC1,ACC2,ACC3
count,6.448400e+06,6448400.0,6448400.0,6.448400e+06,6.448400e+06,6.448400e+06,6.448400e+06,6.448400e+06
mean,7.078980e-04,-1.5,0.0,-2.731500e+02,4.976702e-02,8.307913e-01,-6.681915e-02,-3.551541e-01
std,2.942117e-01,0.0,0.0,0.000000e+00,3.311963e+00,1.819352e-01,8.429531e-02,2.320015e-01
min,-1.499908e+00,-1.5,0.0,-2.731500e+02,-2.011871e+01,-6.956000e-01,-1.483000e+00,-4.249000e+00
25%,-1.335754e-01,-1.5,0.0,-2.731500e+02,-1.930237e+00,7.448000e-01,-1.026000e-01,-5.622000e-01
50%,-6.166077e-02,-1.5,0.0,-2.731500e+02,3.509521e-02,8.386000e-01,-7.539999e-02,-3.374000e-01
75%,5.351257e-02,-1.5,0.0,-2.731500e+02,1.853943e+00,9.028000e-01,-3.439999e-02,-1.924000e-01
max,1.499954e+00,-1.5,0.0,-2.731500e+02,2.664185e+01,3.951200e+00,1.244200e+00,2.957800e+00


Observations : 
EMG,Temp et EDA semblent être les mêmes valeurs dans tout le dataset.

In [28]:
df_chest.EMG.value_counts()

-1.5    6448400
Name: EMG, dtype: int64

In [29]:
df_chest.EDA.value_counts()

0.0    6448400
Name: EDA, dtype: int64

In [30]:
df_chest.Temp.value_counts()

-273.149994    6448400
Name: Temp, dtype: int64

###Wrist

In [0]:
df_wrist = pd.DataFrame()
for i in df.get('signal').get('wrist').keys():
  column = df.get('signal').get('wrist').get(i)
  if(i == 'ACC'):
    name_column = ['ACC1','ACC2','ACC3']
  else:
    name_column = [i]
  column = pd.DataFrame(column, columns=name_column)
  df_wrist = pd.concat((df_wrist,column), axis=1)


In [53]:
df_wrist

,ACC1,ACC2,ACC3,BVP,EDA,TEMP
0,-0.562500,0.046875,0.84375,-4.36,0.267766,31.67
1,-0.562500,0.046875,0.84375,-5.11,0.262641,31.67
2,-0.562500,0.046875,0.84375,-5.40,0.261360,31.67
3,-0.578125,0.046875,0.84375,-5.19,0.260079,31.67
4,-0.562500,0.046875,0.84375,-4.59,0.261360,31.69
...,...,...,...,...,...,...
525115,NaN,NaN,NaN,6.95,NaN,NaN
525116,NaN,NaN,NaN,4.90,NaN,NaN
525117,NaN,NaN,NaN,2.64,NaN,NaN
525118,NaN,NaN,NaN,0.17,NaN,NaN


In [55]:
df_wrist.describe()

,ACC1,ACC2,ACC3,BVP,EDA,TEMP
count,262560.000000,262560.000000,262560.000000,525120.000000,32820.000000,32820.000000
mean,-0.492370,0.291052,0.421304,-0.000448,8.081753,33.492074
std,0.324609,0.608959,0.381986,37.960108,5.968085,1.923040
min,-2.000000,-2.000000,-2.000000,-648.580000,0.245986,28.030000
25%,-0.734375,-0.078125,0.156250,-16.250000,4.249697,31.790000
50%,-0.484375,0.015625,0.453125,1.320000,6.810801,34.250000
75%,-0.281250,0.875000,0.718750,16.410000,11.175568,35.030000
max,1.984375,1.984375,1.984375,482.770000,29.002420,35.810000


In [0]:
#fusion des datas sur les individus
questionnaire = pd.DataFrame()
for i in range(1,16):
  chemin = "./PPG_FieldStudy/S"+str(i)+"/S"+str(i)+".pkl"
  print(chemin)
  df = pd.read_pickle(chemin)
  questionnaire = questionnaire.append(pd.DataFrame(data=[df['questionnaire']]))

./PPG_FieldStudy/S2/S2.pkl
./PPG_FieldStudy/S3/S3.pkl
./PPG_FieldStudy/S4/S4.pkl
./PPG_FieldStudy/S5/S5.pkl
./PPG_FieldStudy/S6/S6.pkl
./PPG_FieldStudy/S7/S7.pkl
./PPG_FieldStudy/S8/S8.pkl
./PPG_FieldStudy/S9/S9.pkl
./PPG_FieldStudy/S10/S10.pkl
./PPG_FieldStudy/S11/S11.pkl
./PPG_FieldStudy/S12/S12.pkl
./PPG_FieldStudy/S13/S13.pkl
./PPG_FieldStudy/S14/S14.pkl
./PPG_FieldStudy/S15/S15.pkl


In [0]:
questionnaire.reset_index(drop = True, inplace = True)
questionnaire

,WEIGHT,Gender,AGE,HEIGHT,SKIN,SPORT
0,78.0,m,34,182.0,3,6
1,80.0,m,28,189.0,3,5
2,60.0,m,25,170.0,3,5
3,57.0,m,25,168.0,4,5
4,70.0,f,21,180.0,3,4
5,70.0,f,37,176.0,3,1
6,58.0,f,21,168.0,3,2
7,70.0,m,43,179.0,3,5
8,60.0,f,28,167.0,4,5
9,56.0,f,55,164.0,4,5


In [23]:
#Ils ne font pas tous la même taille
rpeaks_all = pd.DataFrame()
for i in range(1,3):
  chemin = "./PPG_FieldStudy/S"+str(i)+"/S"+str(i)+".pkl"
  print(chemin)
  df = pd.read_pickle(chemin)
  print(len(df['rpeaks']))
  rpeaks_all[df['subject']]= df['rpeaks']

./PPG_FieldStudy/S1/S1.pkl
11431
./PPG_FieldStudy/S2/S2.pkl
11247


ValueError: ignored

‘activity’: includes the activity labels, providing IDs 0…8 (see activity list in Section II). These
activity labels are based on the file SX_activity.csv. However, in order to make further
processing of the dataset more convenient, an activity-signal with 4 Hz sampling rate (which
is the lowest sampling rate across all recorded raw sensor data) was created.
- ‘label’: includes the ground truth heart rate information. As described above, this is provided
as the mean of the ECG-based instantaneous heart rate, given on a sliding window of 8
seconds, shifted with 2 seconds.
- ‘questionnaire’: includes information about the subject, extracted from SX_quest.csv. Details
were given above, in Section I.4.
- ‘rpeaks’: the index of the identified and corrected R-peaks, referring to the ECG-signal. As
described above, the identified R-peaks provide the basis of the heart rate ground truth.
- ‘signal’: includes all the synchronised raw data, in two fields:
o ‘chest’: RespiBAN data (all the modalities: ACC, ECG, EDA, EMG, RESP, TEMP). As
mentioned above, the modalities ‘EDA’, ‘EMG’ and ‘Temp’ only include dummy data
and should thus be ignored.
o ‘wrist’: Empatica E4 data (all the modalities: ACC, BVP, EDA, TEMP)
- ‘subject’: the current subject’s ID


In [0]:
#Plot des activités
for i in range(1,16):
  chemin = "./PPG_FieldStudy/S"+str(i)+"/S"+str(i)+".pkl"
  print(chemin)
  df = pd.read_pickle(chemin)
  plot_activity(df)

In [0]:
dataframe = df.get('signal').get('chest')
df_signal = pd.DataFrame([dataframe])
df_signal['Temp']
#Divisé en deux : Chest et waist

In [0]:
#Pour chest
df_signal = pd.DataFrame()
for i in df.get('signal').get('chest').keys():
  column = df.get('signal').get('chest').get(i)
  da[i] = column.tolist()

In [0]:
#Pour 

In [0]:
#A séparer en trois colonnes
da['ACC'][0]